Nous avons réaliser ce modèle afin de classifier les avis des doneurs soient positifs, soient négatifsa dans le but pouvoir recommander par la suite aux donneurs les projets qui ont apprécié la majorité de la communauté qui utilise notre plateforme.

In [213]:
import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import  matplotlib.pyplot as plt

from sklearn.model_selection import KFold
from tokenizers import BertWordPieceTokenizer

import warnings 
warnings.simplefilter('ignore')

In [198]:
!pip install -q transformers --upgrade
!pip show transformers

Name: transformers
Version: 4.1.1
Summary: State-of-the-art Natural Language Processing for TensorFlow 2.0 and PyTorch
Home-page: https://github.com/huggingface/transformers
Author: Thomas Wolf, Lysandre Debut, Victor Sanh, Julien Chaumond, Sam Shleifer, Patrick von Platen, Sylvain Gugger, Google AI Language Team Authors, Open AI team Authors, Facebook AI Authors, Carnegie Mellon University Authors
Author-email: thomas@huggingface.co
License: Apache
Location: c:\users\msi\anaconda3\lib\site-packages
Requires: filelock, tqdm, numpy, sacremoses, requests, tokenizers, packaging, regex
Required-by: 


In [199]:
!pip install tensorflow==2.2.O

ERROR: Could not find a version that satisfies the requirement tensorflow==2.2.O (from versions: 2.2.0rc1, 2.2.0rc2, 2.2.0rc3, 2.2.0rc4, 2.2.0, 2.2.1, 2.3.0rc0, 2.3.0rc1, 2.3.0rc2, 2.3.0, 2.3.1, 2.4.0rc0, 2.4.0rc1, 2.4.0rc2, 2.4.0rc3, 2.4.0rc4, 2.4.0)
ERROR: No matching distribution found for tensorflow==2.2.O


In [200]:
#data = pd.read_csv('../input/vneuron/preprocessed_texts_2700row.csv',index_col = [0])#

In [201]:
#data.intent = data.intent.map({'positive' :  1 , 'negative' : 0})

In [202]:
data = pd.read_csv('./final_data.csv')

In [203]:
data

,text,intent
0,0000000000000000000 nulle,0.0
1,marzen mleyektou,0.0
2,يا سي امين ممسطك وليت والله بففففففففففف,0.0
3,M3alem hhhhhhh,1.0
4,عسل حسين عسل,1.0
...,...,...
20334,الحق موش فيها الحق في اللي يطرح في ها المواضيع...,0.0
20335,Mamestek w mabslek w marzen mlayaktek,0.0
20336,وليد..كل يوم حبك يزيد..,1.0
20337,هلكك ربي ياكلبة وضهر للناس حقيقتك يا واطية الل...,0.0


In [204]:
EPOCHS = 5
BATCH_SIZE = 32 
MAX_LEN = 192
#MODEL = 'jplu/tf-xlm-roberta-large'
AUTO = tf.data.experimental.AUTOTUNE

In [258]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=512):

    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding()
    all_ids = []
    
    for i in tqdm(range(0, len(texts), chunk_size)):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

In [259]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
#         return_attention_masks=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        truncation=True,
        max_length=maxlen
        )


    
    return np.array(enc_di['input_ids'])

In [262]:
#tokenizer = AutoTokenizer.from_pretrained(MODEL)

tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
# Save the loaded tokenizer locally
tokenizer.save_pretrained('.')
# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=False)
print(type(fast_tokenizer))

<class 'tokenizers.implementations.bert_wordpiece.BertWordPieceTokenizer'>


In [263]:
def build_model(transformer, max_len=512):

    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=6e-6), loss='binary_crossentropy', metrics=['accuracy','AUC'])
    
    return model

In [264]:
# texts = regular_encode(data.text.values, tokenizer, maxlen=MAX_LEN)
texts = fast_encode(data.text.values.astype(str), fast_tokenizer, maxlen=MAX_LEN)
ys = data.intent.values

In [265]:
# batch = tokenizer(batch_sentences, batch_of_second_sentences, padding='max_length', truncation='longest_first',max_length=MAX_LEN, return_tensors="pt")

In [266]:
# MAX_LEN

In [267]:
def create_train(x_train,y_train) :
    train_dataset = (
        tf.data.Dataset
        .from_tensor_slices((x_train, y_train))
        .repeat()
        .shuffle(2048)
        .batch(BATCH_SIZE)
        .prefetch(AUTO)
    )
    return  train_dataset

def create_valid(x_valid,y_valid) :
    valid_dataset = (
        tf.data.Dataset
        .from_tensor_slices((x_valid, y_valid))
        .batch(BATCH_SIZE)
        .cache()
        .prefetch(AUTO)
    )
    
    return valid_dataset

def create_test(x_test) :
    test_dataset = (
        tf.data.Dataset
        .from_tensor_slices(x_test)
        .batch(BATCH_SIZE)
    )
    return test_dataset

In [268]:
print(len(texts))

20339


In [271]:
from sklearn.model_selection import KFold

FOLDS = 5
SEED  = 42

skf = KFold(n_splits=FOLDS,shuffle=True,random_state=SEED)
print(skf.split(texts,ys))
for fold,(train_indices,valid_indices) in enumerate(skf.split(texts,ys)) :
    print() 
    print('#'*25)
    print('Fold' , fold+1)
    print('#'*25)
    
    #transformer_layer = TFAutoModel.from_pretrained(MODEL)
    transformer_layer = (
        transformers.TFDistilBertModel
        .from_pretrained('distilbert-base-multilingual-cased')
    )
    model = build_model(transformer_layer, max_len=MAX_LEN)
    
    sv = tf.keras.callbacks.ModelCheckpoint(
        'fold-%i.h5'%fold, monitor='val_loss', verbose=0, save_best_only=True,
        save_weights_only=True, mode='min', save_freq='epoch')
    
    n_steps = train_indices.shape[0] // BATCH_SIZE
    history = model.fit(
    create_train(texts[train_indices],ys[train_indices]),
    steps_per_epoch=n_steps,
    validation_data=create_valid(texts[valid_indices],ys[valid_indices]),
    epochs=EPOCHS,
    callbacks =  [sv]    
    )
    
    
    plt.figure(figsize=(15,5))
    plt.plot(np.arange(EPOCHS),history.history['auc'],'-o',label='Train AUC',color='#ff7f0e')
    plt.plot(np.arange(EPOCHS),history.history['val_auc'],'-o',label='Val AUC',color='#1f77b4')
    x = np.argmax( history.history['val_auc'] ); y = np.max( history.history['val_auc'] )
    xdist = plt.xlim()[1] - plt.xlim()[0]; ydist = plt.ylim()[1] - plt.ylim()[0]
    plt.scatter(x,y,s=200,color='#1f77b4'); plt.text(x-0.03*xdist,y-0.13*ydist,'max auc\n%.2f'%y,size=14)
    plt.ylabel('AUC',size=14); plt.xlabel('Epoch',size=14)
    plt.legend(loc=2)
    plt2 = plt.gca().twinx()
    plt2.plot(np.arange(EPOCHS),history.history['loss'],'-o',label='Train Loss',color='#2ca02c')
    plt2.plot(np.arange(EPOCHS)<,history.history['val_loss'],'-o',label='Val Loss',color='#d62728')
    x = np.argmin( history.history['val_loss'] ); y = np.min( history.history['val_loss'] )
    ydist = plt.ylim()[1] - plt.ylim()[0]
    plt.scatter(x,y,s=200,color='#d62728'); plt.text(x-0.03*xdist,y+0.05*ydist,'min loss',size=14)
    plt.ylabel('Loss',size=14)
    plt.title('FOLD %i Distilbert-base-multilingual-cased'%
                (fold+1),size=18)
    plt.legend(loc=3)
    plt.show()  

<generator object _BaseKFold.split at 0x000001E78D0AA820>

#########################
Fold 1
#########################


Some layers from the model checkpoint at distilbert-base-multilingual-cased were not used when initializing TFDistilBertModel: ['vocab_projector', 'vocab_transform', 'activation_13', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Unable to locate the source code of <bound method TFDistilBertModel.call of <transformers.modeling_tf_distilbert.TFDistilBertModel object at 0x000001E7A7FAF580>>. Note that functions defined in certain environments, like the interactive Python shell do not expose their source code. If that is the case, you should to define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Unable to locate the source code of <bound method TFDistilBertModel.call of <tr

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Unable to locate the source code of <bound method TFTransformerBlock.call of <transformers.modeling_tf_distilbert.TFTransformerBlock object at 0x000001E7E5D72820>>. Note that functions defined in certain environments, like the interactive Python shell do not expose their source code. If that is the case, you should to define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Unable to locate the source code of <bound method TFMultiHeadSelfAttention.ca

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

# Testing : 

In [ ]:
test = pd.read_csv('./final_data.csv')

test_texts = fast_encode(test.text.values.astype(str), fast_tokenizer, maxlen=MAX_LEN)

test_ys = test.intent.values

In [ ]:
results = model.predict(create_test(test_texts))
for i,result in enumerate(results) :
    if result > 0.5 :
        results[i] = 1
    else :
        results[i] = 0

In [ ]:
confusion_matrix(test_ys, results)

In [ ]:
print(classification_report(test_ys, results))

In [ ]:
all_ids = []
encs = fast_tokenizer.encode_batch(['j adore ','khedmtkom behyaa barcha','service khayeb'])
all_ids.extend([enc.ids for enc in encs])


test_data = create_test(np.array(all_ids))

predictions = model.predict(test_data)
#print(predictions*10)
for prediction in predictions :
    print(prediction)

#  Wrong Predictions : 

In [ ]:
for i,prediction in enumerate(results) :
    if prediction != test_ys[i] :
        print(test['text'][i])

# Saving the model : 

In [ ]:
import pickle


# Save the Modle to file in the current working directory

Pkl_Filename = "pickled_model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model, file)

# Semi supervized Pseudo Labelling : 

In [ ]:
extra_data = pd.read_csv('../input/vneuron/oreedoo_data.csv')

In [ ]:
l = []
for i in range(len(extra_data)) : 
    l.append(str(extra_data['r'][i]))

In [ ]:
model.load_weights('fold-2.h5')

In [ ]:
all_ids = []
encs = fast_tokenizer.encode_batch(l)
all_ids.extend([enc.ids for enc in encs])


test_data = create_test(np.array(all_ids))

predictions = model.predict(test_data)
#print(predictions*10)
for prediction in predictions :
    print(prediction)

In [ ]:
to_use_label_1 = []
to_use_label_0 = []
for i,prediction in enumerate(predictions) :
    if prediction > 0.8 :
        to_use_label_1.append(i)
    if prediction < 0.001 :
        to_use_label_0.append(i)

In [ ]:
print('Added positive labels :',len(to_use_label_1))
print('Added negative labels :',len(to_use_label_0))

In [ ]:
print('Added Positive examples :')
for i in to_use_label_1 :
    print(extra_data['r'][i])   

In [ ]:
print('Added Negative examples :')
for i in to_use_label_0 :
    print(extra_data['r'][i]) 